# Download and prepare NEMO data

We'll download NEMO example data from https://doi.org/10.5281/zenodo.3755516.
As these netCDF files come with a record dimension, and as record dimensions come with performance penaldies, we create a copy that only has fixed dimensions.

In [ ]:
# Parameters
orig_data_dir = "/data/parcels_v2.2_nemo_tests_data/orig/"
fixtime_data_dir = "/data/parcels_v2.2_nemo_tests_data/fixtime/"
zenodo_record = 3755517

In [ ]:
# Get proxy settings from /etc/environment.
import os

with open("/etc/environment") as f:
    for line in f.readlines():
        if "http_proxy" in line or "https_proxy" in line:
            key, val = tuple(line.strip().split("="))
            val = val.strip("'\"\n")
            os.environ[key] = val

In [ ]:
# Download all files for the Zenodo record

from pathlib import Path
import requests
from urllib.parse import urlparse

def download_record_to(record_id, target_dir):
    target_dir = Path(target_dir)
    target_dir.mkdir(exist_ok=True, parents=True)
    record = requests.get(url=f"https://zenodo.org/api/records/{record_id}").json()
    urls  = [f["links"]["self"] for f in record["files"]]
    targets = [target_dir / Path(urlparse(url).path).name for url in urls]
    for url, target in zip(urls, targets):
        if not target.exists():
            !curl -o {str(target)} {str(url)}
        else:
            print(f"{target} exists")
    return targets

files = download_record_to(zenodo_record, orig_data_dir)

In [ ]:
# Create fixed-dim directory and convert data files.

Path(fixtime_data_dir).mkdir(parents=True, exist_ok=True)

nc_files = Path(orig_data_dir).glob("*.nc")
for ncf in nc_files:
    target_file = Path(fixtime_data_dir) / ncf.name
    print(f"will convert {str(ncf.name)} ...")
    !nccopy -u -7 {str(ncf)} {str(target_file)}  # fixed time dim and netcdf4-classic
    print(f"... {str(ncf.name)} done")